In [ ]:
import teradatasql
import pandas as pd


# you can connect directly to GDW table downloaded or use csv if you are not connected to the network

# download centre mapping from GDW
with teradatasql.connect(host='tdp5dbcw.teradata.westpac.com.au', user='M118954', password='mygdw2021!') as connect:
    lg_hier = pd.read_sql('select * from dwpviewa.LG_Hier_Centre where date between from_date and to_date and lg_centre_l14_key in (\'RB4892\',\'RB4152\',\'RB7062\')', connect)

# download product mapping from GDW
with teradatasql.connect(host='tdp5dbcw.teradata.westpac.com.au', user='M118954', password='mygdw2021!') as connect:
    lg_prod = pd.read_sql('select * from dwpviewa.LG_Hier_Product where date between from_date and to_date', connect)
    
# download the latest credit risk file and dont forget to change the month
with teradatasql.connect(host='tdp5dbcw.teradata.westpac.com.au', user='M118954', password='mygdw2021!') as connect:
    df = pd.read_sql('select * from finiq.CRDBEXTRACT where month_key in (\'202009\')', connect)


# create mapping table to csv
lg_hier.to_csv(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\Map_Centre.csv')
lg_prod.to_csv(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\Map_Product.csv')



In [11]:
import teradatasql
import pandas as pd

# download the latest credit risk file and dont forget to change the month
with teradatasql.connect(host='tdp5dbcw.teradata.westpac.com.au', user='M118954', password='mygdw2021!') as connect:
    df = pd.read_sql(mysql, connect)


OperationalError: [Version 17.0.0.5] [Session 17976675] [Teradata SQL Driver] Failure sending Logoff Request message
 at gosqldriver/teradatasql.(*teradataConnection).makeDriverError TeradataConnection.go:1101
 at gosqldriver/teradatasql.(*teradataConnection).sendAndReceive TeradataConnection.go:1387
 at gosqldriver/teradatasql.(*teradataConnection).Close TeradataConnection.go:1883
 at database/sql.(*driverConn).finalClose.func2 sql.go:594
 at database/sql.withLock sql.go:3204
 at database/sql.(*driverConn).finalClose sql.go:592
 at database/sql.(*DB).Close sql.go:849
 at main.goCloseConnection goside.go:418
 at main._cgoexpwrap_7f5c3249bf12_goCloseConnection _cgo_gotypes.go:285
 at runtime.cgocallbackg1 cgocall.go:332
 at runtime.cgocallbackg cgocall.go:207
 at runtime.cgocallback_gofunc asm_amd64.s:793
 at runtime.goexit asm_amd64.s:1373
Caused by write tcp 10.37.111.138:52189->10.39.106.43:1025: wsasend: An existing connection was forcibly closed by the remote host.

In [10]:
mysql = '''select * from finiq.CRDBEXTRACT where month_key in (\'202009\')'''


In [9]:
mysql

"select * from\nfiniq.CRDBEXTRACT a\nleft join finiq.TM1_CtrAggOld agg\non agg.lg_centre_id = a.centre_id\nwhere month_key in ('202011')"

In [82]:
# download centre mapping from GDW
with teradatasql.connect(host='tdp5dbcw.teradata.westpac.com.au', user='M118954', password='mygdw2021!') as connect:
    lg_hier = pd.read_sql('select * from dwpviewa.LG_Hier_Centre where date between from_date and to_date and lg_centre_l14_key in (\'RB4892\',\'RB4152\',\'RB7062\')', connect)


In [3]:
def leadZero(a):
    mystr = str(a)
    if len(mystr) == 5:
        mystr = '0' + mystr
    if len(mystr) == 4:
        mystr = '00' + mystr
    if len(mystr) == 3:
        mystr = '000' + mystr
    else:
        mystr = mystr
    return mystr

# ECC Only
def CR_ECC(Class,ECC,RWA):
    if Class in ['SMECORP','SLIPRE','CORP']:
        calc = ECC
    else:
        calc = 0
    return calc

# RWA Only
def CR_RWA(Class,ECC,RWA):
    if Class in ['SMECORP','SLIPRE','CORP']:
        calc = 0
    else:
        calc = RWA * 0.0875
    return calc

# Total Credit Risk (ECC + RWA)
def CR_Tot(Class,ECC,RWA):
    if Class in ['SMECORP','SLIPRE','CORP']:
        calc = ECC
    else:
        calc = RWA * 0.0875
    return calc

In [194]:
import pandas as pd
import time


start = time.time()
# download the file from spreadsheet
risk = pd.read_excel(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\RECSOURCEDATA.xlsx', sheet_name='RISK DATA') 
coi = pd.read_excel(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\RECSOURCEDATA.xlsx', sheet_name='COI DATA') 

# fill all the NAs
risk['CENTRE_ID'] = risk['CENTRE_ID'].fillna(0)
risk['PRODUCT_CODE'] = risk['PRODUCT_CODE'].fillna(0)

# convert data type
risk = risk.astype({'CENTRE_ID': 'int64','PRODUCT_CODE':'int64'})

finish = time.time()

print ('It took around ' + str((finish-start)/60) + ' minutes')


It took around 2.564243483543396 minutes


In [218]:
coi.head()

,MONTH_KEY,CENTRE_ID,PRODUCT_CODE,ASSET_SUBCLASS,l13,l12,l11,l10,prod10,prod09,prod08,ECC,RWA
0,202009,102275,13255,QRE,Westpac_BSA_BOM,NSW Regional,Westpac NSW Regional,Westpac NSW Regional - Branch,Personal Finance,Personal Finance - Revolving,Personal Credit Cards,12730.85,115224.76
1,202009,115025,11704,MRTG,Westpac_BSA_BOM,QLD,Westpac QLD,Westpac QLD - Branch,Mortgage Finance,Mortgage Variable,HL Var - Term,2416.10,40491.82
2,202009,160557,13211,QRE,Westpac_BSA_BOM,QLD,Westpac QLD,Westpac QLD - Branch,Personal Finance,Personal Finance - Revolving,Personal Credit Cards,6.40,39.49
3,202009,849607,13344,QRE,Westpac_BSA_BOM,VIC/TAS,Bank of Melbourne,BOM Retail & Business,Personal Finance,Personal Finance - Revolving,Personal Credit Cards,12580.80,127826.27
4,202009,116355,41284,OTHRETL,Westpac_BSA_BOM,SANT/WA,Westpac SANT,Westpac SANT - Branch,Personal Deposits,Personal Transactions,Personal Transactions - Limited,0.00,39.74


In [219]:
a = risk[['CENTRE_ID','ASSET_SUBCLASS','DIVISION','BUSINESS_UNIT','PRODUCT_CODE','PRODUCT_1','ECC_AMT','RWA']]
b = coi[['CENTRE_ID','ASSET_SUBCLASS','l13','PRODUCT_CODE','prod10','ECC','RWA']]

finalrec = pd.merge(a,b,on=['CENTRE_ID','PRODUCT_CODE','ASSET_SUBCLASS'],how='left')
finalrec = finalrec[['CENTRE_ID', 'PRODUCT_CODE','ASSET_SUBCLASS','DIVISION', 'BUSINESS_UNIT', 'l13', 'PRODUCT_1', 'prod10','ECC_AMT', 'RWA_x',  'ECC', 'RWA_y']]

finalrec.to_csv((r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\FinalRec.csv'))

In [211]:
['CENTRE_ID', 'DIVISION', 'BUSINESS_UNIT', 'l13','PRODUCT_CODE', 'PRODUCT_1', 'prod10','ECC_AMT', 'RWA_x',  'ECC', 'RWA_y']

Index(['CENTRE_ID', 'DIVISION', 'BUSINESS_UNIT', 'PRODUCT_CODE', 'PRODUCT_1',
       'ECC_AMT', 'RWA_x', 'l13', 'prod10', 'ECC', 'RWA_y'],
      dtype='object')

In [202]:
finalrec[['CENTRE_ID', 'DIVISION', 'BUSINESS_UNIT', 'PRODUCT_CODE_x', 'PRODUCT_CODE_y', 'PRODUCT_1','prod10','ECC_AMT', 'RWA_x','ECC', 'RWA_y']]

,CENTRE_ID,DIVISION,BUSINESS_UNIT,PRODUCT_CODE_x,PRODUCT_CODE_y,PRODUCT_1,prod10,ECC_AMT,RWA_x,ECC,RWA_y
0,849243,Commercial Business Banking,Commercial Banking,0,15457.0,Business,Business Finance,37874.82,603137.14,22546.38,411916.40
1,849243,Commercial Business Banking,Commercial Banking,0,15145.0,Business,Business Finance,37874.82,603137.14,1332761.00,28839442.10
2,849243,Commercial Business Banking,Commercial Banking,0,11122.0,Business,Mortgage Finance,37874.82,603137.14,19673.87,674192.17
3,849243,Commercial Business Banking,Commercial Banking,0,41403.0,Business,Business Deposits,37874.82,603137.14,51039.87,790022.29
4,849243,Commercial Business Banking,Commercial Banking,0,13363.0,Business,Business Finance,37874.82,603137.14,33126.10,744600.71
...,...,...,...,...,...,...,...,...,...,...,...
8340417,849407,WIB,FM Debt Markets,17302,NaN,Business,NaN,593.05,1289204.43,NaN,NaN
8340418,849408,WIB,FM Debt Markets,17302,NaN,Business,NaN,17212.32,1289204.43,NaN,NaN
8340419,230127,WIB,WIB Other,13126,NaN,Credit Card,NaN,72.32,1289204.43,NaN,NaN
8340420,230127,WIB,WIB Other,13332,NaN,Credit Card,NaN,37.10,1289204.43,NaN,NaN


In [4]:
import time
import pandas as pd

start = time.time()
lg_hier = pd.read_csv(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\Map_Centre.csv')
lg_hier['LG_Centre_Id'] = lg_hier['LG_Centre_Id'].str.strip()    # remove trailing spaces


aggMap = pd.read_excel(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\Check Credit Risk.xlsm', sheet_name='Ctr Mapping') 
#aggMap = aggMap.rename(columns={'CENTRE_ID':'LG_Centre_Id'})

# convert to string
aggMap = aggMap.astype({'Centre': 'str'})
aggMap = aggMap.astype({'Mapping': 'str'})

# rename column
aggMap = aggMap.rename(columns={'Centre':'LG_Centre_Id'})
aggMap = aggMap[['LG_Centre_Id','Mapping']]

# remove trailing spaces
aggMap['LG_Centre_Id'] = aggMap['LG_Centre_Id'].str.strip()    # remove trailing spaces
aggMap['Mapping'] = aggMap['Mapping'].str.strip()    # remove trailing spaces
aggMap['LG_Centre_Id'] = aggMap.apply(lambda x:leadZero(x['LG_Centre_Id']),axis=1)    # apply function to fix leading zero


# run merging sql
aggMap = pd.merge(aggMap,lg_hier,how="left",on=['LG_Centre_Id'])
aggMap = aggMap[['LG_Centre_Id','Mapping','LG_Centre_L14_Name','LG_Centre_L13_Name','LG_Centre_L12_Name']]


#rename columns
aggMap = aggMap.rename(columns={'LG_Centre_L14_Name':'L14',
                                'LG_Centre_L13_Name':'L13',
                                'LG_Centre_L12_Name':'L12',
                                'LG_Centre_Id':'Centre_Id',
                                'Mapping':'LG_Centre_Id'
                               })

# run second mergin sql
aggMap = pd.merge(aggMap,lg_hier,how="left",on=['LG_Centre_Id'])
aggMap = aggMap[['Centre_Id','LG_Centre_Id','L14','L13','L12','LG_Centre_L14_Name','LG_Centre_L13_Name','LG_Centre_L12_Name']]

#rename columns 2
aggMap = aggMap.rename(columns={'LG_Centre_L14_Name':'L14_Agg',
                                'LG_Centre_L13_Name':'L13_Agg',
                                'LG_Centre_L12_Name':'L12_Agg',
                                'LG_Centre_Id':'Agg_Id'
                               })



finish = time.time()
duration = round(((finish-start)/60),2)


if (finish-start)<60:
    duration = round((finish-start),2)
    print ('it took around ' + str(duration) + ' seconds')
else:
    duration = round(((finish-start)/60),2)
    print ('it took around ' + str(duration) + ' minutes')

    

it took around 55.81 seconds


In [6]:
aggMap.to_csv(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\OldData.csv')